## Extract the kardex de accionistas execl file from mongodb and parse into a json fin another colletion
#### Connect to mongodb and make sure we have access to the excel files

In [75]:
packages = ['pandas', 'pymongo', 'gridfs', 'concurrent.futures']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

from pymongo import MongoClient
from gridfs import GridFS
from concurrent.futures import ThreadPoolExecutor
# we run this so that we can access relative paths
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path: sys.path.append(parent_dir)
from functions.GridFsFileOperations import GridFsFileOperations

endpoint = '10.0.10.5:27017'
#endpoint = '192.168.1.10:27017'
database = 'supercias'
collection = 'kradex_de_accioniestas'
path = '../storage/kardek_de_accionistas/'

# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]
# Create instance of GridFsFileOperations
filesOps = GridFsFileOperations(db, collection, path)

Read download every file and parse data into mongodb

First let's define the parser

In [76]:
import pandas as pd
import unicodedata
def parse_excel(file_path):
    file_size = os.path.getsize(file_path)
    # Convert the size to a human-readable format (optional)
    print('File size in bytes:', file_size)
    if(file_size <= 100): return 
    df = pd.read_excel(file_path)
    # get the ruc from the filename
    ruc = file_path.split('.')[-2].split('/')[-1]
    # get the comapny name
    name = df.iloc[0,0]
    df.columns = df.iloc[3]
    # Remove accents from column names
    df.columns = [ unicodedata.normalize('NFKD', col).encode('ascii', 'ignore').decode('utf-8') for col in df.columns ]
    df = df[3:].reset_index(drop=True) 
    # remove the first column
    df = df.drop(df.columns[0], axis=1)  
    df = df.dropna(subset=["IDENTIFICACION"])
    # add the rows for NOMBRE COMPANIA and RUC
    df['RUC'] = ruc
    df['NOMBRE DE COMPANIA'] = name
    print('ruc:', ruc, 'nombre', name)
    return df

In [80]:
def upload_rows(df, collection):
    collection.insert_many(df.to_dict('records'))


In [83]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
import os
from pymongo import MongoClient
# read execl fil into pd dataframe
path = '../../storage/kardex_de_accionistas/'
file_i = 6
# Connect to MongoDB
endpoint = '10.0.10.5:27017' # use vpn
client = MongoClient("mongodb://"+endpoint)
db = client["supercias"] 
collection = db["kardex_de_accionistas"]  
# List all files in the directory
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
for file in files:
    df = parse_excel(path + file)
    if(df is None): continue
    else:
        upload_rows(df, collection)
        print (f"Uploaded {file}")

File size in bytes: 15628
ruc: 2490400446001 nombre NATUR CORP S.A.S.
Uploaded 2490400446001.xlsx
File size in bytes: 15640
ruc: 2490402686001 nombre COMERCIALIZADORA DE PRODUCTOS DEL MAR Y AFINES ADONAI S.A.S.
Uploaded 2490402686001.xlsx
File size in bytes: 15631
ruc: 2490407211001 nombre SHRIMP WORLD ALEXMAR S.A.S.
Uploaded 2490407211001.xlsx
File size in bytes: 15622
ruc: 2490400292001 nombre ALCARSAN SERVICE S.A.S.
Uploaded 2490400292001.xlsx
File size in bytes: 15942
ruc: 2490406371001 nombre SAVINLAB S.A.
Uploaded 2490406371001.xlsx
File size in bytes: 15631
ruc: 2490402384001 nombre LUBRIREPUESTOS-MULTIMARK S.A.S.
Uploaded 2490402384001.xlsx
File size in bytes: 15717
ruc: 2490405677001 nombre SVCONTSAS SOCIEDAD POR ACCIONES SIMPLIFICADA
Uploaded 2490405677001.xlsx
File size in bytes: 15850
ruc: 2490407882001 nombre LEGALTRAINERS S.A.S.
Uploaded 2490407882001.xlsx
File size in bytes: 15786
ruc: 2490400233001 nombre JOSADRIZ C.LTDA.
Uploaded 2490400233001.xlsx
File size in bytes: 